In [6]:
import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq
import pyarrow.compute as pc
import numpy as np
from pyarrow import csv
from pathlib import Path
from rich.progress import track

In [ ]:
!ls -l /data/aiomics/massspec_cache/uniprot/cho/*.parquet
!ls -l /data/aiomics/massspec_cache/uniprot/cho/*.arrow


In [ ]:
for input in track(Path('/data/search_asms2023/cho').glob('*.arrow')):
    output = input.with_suffix('.parquet')
    table = feather.read_table(input)
    pq.write_table(table, output)

In [ ]:
table = pq.read_table("/data/aiomics/massspec_cache/uniprot/cho/predicted_cho_uniprot_tryptic_2_0.parquet")
table

In [2]:
query_table = pq.read_table("/home/djs10/asms2023/test.parquet")
query_table

pyarrow.Table
id: uint64
charge: int8
ev: double
instrument: string
instrument_type: string
instrument_model: string
ion_mode: string
ionization: string
name: string
synonyms: string
scan: string
nce: double
collision_energy: double
retention_time: double
collision_gas: string
insource_voltage: int64
sample_inlet: string
intensity: large_list<item: double>
  child 0, item: double
stddev: large_list<item: double>
  child 0, item: double
product_massinfo: struct<tolerance: double, tolerance_type: dictionary<values=string, indices=int32, ordered=0>, mass_type: dictionary<values=string, indices=int32, ordered=0>, neutral_loss: string, neutral_loss_charge: int64, evenly_spaced: bool>
  child 0, tolerance: double
  child 1, tolerance_type: dictionary<values=string, indices=int32, ordered=0>
  child 2, mass_type: dictionary<values=string, indices=int32, ordered=0>
  child 3, neutral_loss: string
  child 4, neutral_loss_charge: int64
  child 5, evenly_spaced: bool
mz: large_list<item: double>


In [4]:
len(query_table)

143044

In [3]:
mods = query_table["mod_names"]
np_mods = mods.combine_chunks().to_numpy(zero_copy_only=False)
mask = pa.array(~np.array([ np.any(x==737) for x in np_mods ]))
no_tmt_table = query_table.filter(mask)
len(no_tmt_table)

116548

In [12]:
cho_names = csv.read_csv('/home/djs10/asms2023/cho_names_uniq.txt',read_options=csv.ReadOptions(column_names=['name']), parse_options=csv.ParseOptions(delimiter='\t'))
len(cho_names)

110374

In [18]:
cho_set = set(cho_names['name'].to_pylist())
cho_mask = [x in cho_set for x in no_tmt_table['name'].to_pylist()]
print(cho_mask.count(True),cho_mask.count(False))
cho_table = no_tmt_table.filter(cho_mask)
len(cho_table)

8819 107729


8819

In [19]:

pq.write_table(cho_table,"/home/djs10/asms2023/test_filtered.parquet")